## AnkiPy

Python library for creating and managing Anki decks using an excel file as database of cards

Author: Mario Mañana  
Last update: 28/8/2024


packages

In [7]:
import pandas as pd

### Flags

In [8]:
# flags
# use .- Set target: 1.- Proyecto de innovacion docente; 2.- UC24
use = 2

if use == 1: # innovación docente
    path = 'E:\\mario\trabajos2\\innovación_docente_2024\\ankipy\\'
    excel_file = 'uc_tc.xlsx'
    csv_out = 'uc_tc.csv' # default name
elif use == 2: # uc24
    path = 'E:\\mario\\UC24\\documentacion\\'
    excel_file = 'uc24_anki.xlsx'
    csv_out = 'uc24.csv'
else: # unknown value
    print('Set target: error. Unknown value...')



baggregated = True
boverwrite = False 


if use == 1: # innovación docente
    #sheet_names = ['intro', 'ca', 'resolucion', 'trifasica', 'transitorio', 'cuadripolos', 'bobinas', 'filtros', 'rns']
    sheet_names = ['intro', 'ca']
elif use == 2: # uc24
    sheet_names = ['eadmi', 'presupuesto', 'infraestructuras', 'LOSU', 'academico', 'CifrasUC']        # Name of the sheets to read
    #sheet_names = ['eadmi']        # Name of the sheets to read
    

# asignatura. Set 'uc24' for UC 24  
asignatura = 'UC24'

# columns
columns = ['slide', 'pregunta', 'imagen_frontal', 'respuesta', 'imagen_respuesta', 'r1', 'r2', 'r3', 'r4', 'r5', 'r6', 'r7', 'r8', 'r9', 'r10', 'tags','deck',asignatura] # List of columns to read

subdesk_uc24 = {
    "1": "eAdministracion",
    "2": "Presupuesto",
    "3": "Infraestructuras",
    "4": "LOSU",
    "5": "Academico",
    "6": "CifrasUC"
}


csv_aggregated = asignatura + '.csv'

full_excel_path = path + excel_file
print("******************************************")
print(" Input File")
print( full_excel_path)
print("  ")


if baggregated == True:
    full_csv_output = path + csv_aggregated
else:
    full_csv_output = path + sheet + '_00.csv'
print('Output file: ' + full_csv_output)

******************************************
 Input File
E:\mario\UC24\documentacion\uc24_anki.xlsx
  
Output file: E:\mario\UC24\documentacion\UC24.csv


In [9]:
# File header
# More information: https://docs.ankiweb.net/importing/text-files.html 
contenido = """#deck column:17
#separator:Semicolon
#notetype:Basic_Image_several_answers
#columns:slide;pregunta;imagen_frontal;respuesta;imagen_respuesta;r1;r2;r3;r4;r5;r6;r7;r8;r9;r10;tags;deck
#tags column:16
#html:true
"""

# Escribir (sobrescribir si ya existe) el archivo
with open( full_csv_output, "w") as archivo:
    archivo.write(contenido)

# Iterate over the sheet_names array
firsti = True
df_summary = pd.DataFrame(columns=['Subdesk', 'NCards'])

for index, sheet in enumerate( sheet_names):
    #print("-------------------")
    #print("Sheet: " + sheet)
    
    
    df = pd.read_excel( full_excel_path, sheet_name=sheet, usecols=columns, header=0, dtype=str)
    df.columns = df.columns.astype( str)
    
    # remove rows which <asignatura> cell is NaN
    df2 = df[ df[asignatura].notna()]
    
    # set <slide> and <deck> cells   
    for index, row in df2.iterrows():
        df2.at[ index, 'slide'] = sheet + '.' + str(row['slide'])
        if use == 1:
            df2.at[ index, 'deck'] = asignatura + '::' + 'Tema' + str(row[asignatura])  
        elif use == 2:
            df2.at[ index, 'deck'] = asignatura + '::' + subdesk_uc24[ row[asignatura] ]        
        else: #unknown value
            print('Set target: Unknown value...')    
    
           
    nueva_fila = {'Subdesk': [subdesk_uc24[ row[asignatura]]], 'NCards': [len(df2)]}
    df_nuevo = pd.DataFrame( nueva_fila)
    df_summary = pd.concat([df_summary, df_nuevo], ignore_index=True)
    
        #df_nuevo = pd.DataFrame({
        #    'Subdesk': subdesk_uc24[ row[asignatura]], 
        #    'NCards': len(df2)})
        #df_summary = pd.concat([df_summary, df_nuevo], ignore_index=True)       
    # remove <asignatura> column once the dataframe has been filtered   
    dfs = df2.drop( columns=[asignatura])
    dfs.to_csv( full_csv_output, sep=';', index=False, na_rep='  ', mode='a', header=False)
       
    # aggregate all sheets in a single dataframe
    if firsti == True: 
        dfall = dfs
        firsti = False
    else:
        dfall = pd.concat([dfall, dfs], ignore_index=True)
    
    #print(dfs)
    
    
    
# Determinar el ancho de las columnas basándote en los valores más largos
ancho_subdesk = max(len(subdesk) for subdesk in df_summary['Subdesk']) + 10  # Ancho de la columna 'Subdesk'
ancho_cards = max(len(str(card)) for card in df_summary['NCards']) + 5  # Ancho de la columna 'Cards'

print(f"{'Subdesk':<{ancho_subdesk}}{'Cards':<{ancho_cards}}")
print('-' * (ancho_subdesk + ancho_cards + 3))

for subdesk, cards in zip(df_summary['Subdesk'], df_summary['NCards']):
    print(f"{subdesk:<{ancho_subdesk}}{cards:<{ancho_cards}}")

Subdesk                   Cards 
-----------------------------------
eAdministracion           8     
Presupuesto               7     
Infraestructuras          7     
LOSU                      4     
Academico                 3     
CifrasUC                  9     
